In [1]:
import os
import re
import time
import json
import requests
from datetime import datetime

print("Starting document metadata scraping...")

# --- Configuration ---
BASE_API_URL = "https://ncar.gov.sa/api/index.php/api/documents/document-search"
BASE_PDF_URL_PREFIX = "https://ncar.gov.sa/api/index.php/resource/"
BASE_PDF_URL_SUFFIX = "/Documents/OriginalAttachPath"
PDF_OUTPUT_DIR = "NCAR_PDFs"

ITEMS_PER_PAGE = 10
SORT_BY = "approveDate"
SORT_ORDER = "DESC"
START_PAGE = 1 # MODIFY START_PAGE AND END_PAGE, PAGE 1 IS MOST RECENT
END_PAGE = 3 # MODIFY START_PAGE AND END_PAGE, PAGE 1 IS MOST RECENT
REQUEST_DELAY = 0.5  # polite delay

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "Referer": "https://ncar.gov.sa/rules-regulations"
}

BASE_PAYLOAD = {
    "approveTool_id": None,
    "documentCategory_id": None,
    "name": None,
    "ApproveDate": None,
    "PublishingStatus": None,
    "alphabeticalCategory_id": None,
    "alphabeticalSubCategory_id": [],
    "alphabeticalTopic_id": None,
    "approveDate_from": None,
    "approveDate_to": None,
    "generalCategory_id": [],
    "governmentalAgency_childId": [],
    "governmentalAgency_id": None,
    "identical": 1,
    "is_printed": None,
    "is_translated": None,
    "is_valid": None,
    "number": None,
    "omAlQourah_date": None,
    "omAlQourah_version": None,
    "particularCategory_id": [],
    "releaseOrgId": None
}


# --- Helpers ---
def sanitize_filename(name: str, max_length: int = 100) -> str:
    """Clean a string for safe filename use."""
    try:
        name = name.encode('latin1').decode('unicode_escape')
    except (UnicodeEncodeError, UnicodeDecodeError):
        pass
    s = re.sub(r"[^\w\s-]", "", name).strip()
    s = re.sub(r"[\s]+", "-", s)
    return s[:max_length] or "untitled"


def fetch_page(page_num: int, payload: dict) -> dict | None:
    """Fetch a single page of documents."""
    url = f"{BASE_API_URL}/{page_num}/{ITEMS_PER_PAGE}/{SORT_BY}/{SORT_ORDER}"
    print(f"🔎 Requesting page {page_num} ...")
    try:
        response = requests.post(url, headers=HEADERS, json=payload, timeout=15)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"❌ Error fetching page {page_num}: {e}")
        return None


def save_json(data: list, start: int, end: int) -> str:
    """Save scraped data to JSON with page range + formatted date in filename."""
    os.makedirs("output", exist_ok=True)
    today = datetime.now()
    today_str = f"{today.day:02}.{today.month:02}.{today.year}"  # e.g., 09.12.2024
    filename = f"output/ncar_crawl_page{start}-{end}_({today_str}).json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    return filename


def download_pdf(doc: dict, index: int, total: int) -> bool:
    """Download a single PDF if available."""
    encrypted_id = doc.get("id")
    if not encrypted_id:
        print(f"⚠️  Skipping {index+1}/{total}: missing ID")
        return False

    pdf_url = f"{BASE_PDF_URL_PREFIX}{encrypted_id}{BASE_PDF_URL_SUFFIX}"
    title = sanitize_filename(doc.get("title_en", f"document_{index+1}"))
    number = sanitize_filename(doc.get("number", "")) if doc.get("number") else ""
    filename_base = f"{title}_{number}" if number else title

    os.makedirs(PDF_OUTPUT_DIR, exist_ok=True)
    pdf_path = os.path.join(PDF_OUTPUT_DIR, f"{filename_base}.pdf")

    print(f"⬇️  Downloading ({index+1}/{total}): {filename_base}")

    try:
        with requests.get(pdf_url, headers=HEADERS, stream=True, timeout=30) as r:
            r.raise_for_status()
            if "application/pdf" not in r.headers.get("Content-Type", ""):
                print(f"⚠️  Not a PDF ({r.headers.get('Content-Type')}). Skipping.")
                return False

            with open(pdf_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"✅ Saved: {pdf_path}")
        return True

    except requests.RequestException as e:
        print(f"❌ Error downloading '{filename_base}': {e}")
        return False


# --- Scraping Process ---
scraped_data = []
for page in range(START_PAGE, END_PAGE + 1):
    page_data = fetch_page(page, BASE_PAYLOAD)
    if not page_data or not page_data.get("data"):
        print(f"⚠️  No data on page {page}, stopping.")
        break
    scraped_data.extend(page_data["data"])
    time.sleep(REQUEST_DELAY)

print(f"\n📊 Scraping complete. Total documents: {len(scraped_data)}")

if scraped_data:
    json_file = save_json(scraped_data, START_PAGE, END_PAGE)
    print(f"💾 Data saved to: {json_file}")

    print("\n🚀 Starting PDF downloads...")
    downloaded = sum(download_pdf(doc, i, len(scraped_data))
                     for i, doc in enumerate(scraped_data))
    print(f"\n🎉 PDF download complete: {downloaded}/{len(scraped_data)} successful.")
else:
    print("⚠️  No data scraped, skipping PDF downloads.")


Starting document metadata scraping...
🔎 Requesting page 1 ...
🔎 Requesting page 2 ...
🔎 Requesting page 3 ...

📊 Scraping complete. Total documents: 30
💾 Data saved to: output/ncar_crawl_page1-3_(23.09.2025).json

🚀 Starting PDF downloads...
⬇️  Downloading (1/30): آلية-مشاركة-الجهات-الحكومية-في-معارض-الكتاب-الدولية-لعام-1447هـ_24939
✅ Saved: NCAR_PDFs\آلية-مشاركة-الجهات-الحكومية-في-معارض-الكتاب-الدولية-لعام-1447هـ_24939.pdf
⬇️  Downloading (2/30): إضافة-صلاحية-تحديد-المقابل-المالي-لعمل-المرافقين-والمرافقات-للعمالة-الوافدة-النظامية-في-المملكة-لوزي_229
✅ Saved: NCAR_PDFs\إضافة-صلاحية-تحديد-المقابل-المالي-لعمل-المرافقين-والمرافقات-للعمالة-الوافدة-النظامية-في-المملكة-لوزي_229.pdf
⬇️  Downloading (3/30): تعديل-الفقرة-1-من-المادة-4-من-تنظيم-هيئة-المدن-والمناطق-الاقتصادية-الخاصة-لعام-1431هـ_أ76
✅ Saved: NCAR_PDFs\تعديل-الفقرة-1-من-المادة-4-من-تنظيم-هيئة-المدن-والمناطق-الاقتصادية-الخاصة-لعام-1431هـ_أ76.pdf
⬇️  Downloading (4/30): تنظيم-المركز-الوطني-لسلامة-النقل-لعام-1447هـ_212
✅ Saved: NCAR

In [1]:
# ABBYY Vantage — one-cell token + skills check
import os, json, requests

# ==== CONFIG (edit these) ====
CLIENT_ID = "0JwmpC-UcmL3s7KUdlukm_-Rla2pkg"
CLIENT_SECRET = "eJWlhYAPWHsL1fWDjtFDRTNivI2vvgTTHW"
TOKEN_URL = "https://vantage-eu.abbyy.com/auth2/connect/token"
SKILLS_URL = "https://vantage-eu.abbyy.com/api/publicapi/v1/skills"
SCOPE = "openid permissions global.wildcard"
# Optional: corporate proxy (uncomment & set if needed)
# os.environ["HTTPS_PROXY"] = "http://proxy.host:port"

def get_token():
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": SCOPE,
    }
    r = requests.post(TOKEN_URL, data=data, headers=headers, timeout=60)
    try:
        r.raise_for_status()
    except Exception as e:
        print("❌ Token request failed:", e)
        print("Status:", r.status_code, "| Body:", r.text[:500])
        raise
    j = r.json()
    token = j.get("access_token")
    if not token:
        raise RuntimeError(f"Token missing in response: {j}")
    os.environ["ABBYY_TOKEN"] = token
    print(f"✅ Token acquired (expires_in={j.get('expires_in')}s) and saved to os.environ['ABBYY_TOKEN']")
    return token

def call_skills(token):
    r = requests.get(
        SKILLS_URL,
        headers={"Authorization": f"Bearer {token}", "Accept": "application/json"},
        timeout=60,
    )
    ct = r.headers.get("Content-Type", "")
    print("Skills status:", r.status_code)
    if r.status_code == 403:
        print("⚠️  Forbidden (403). This client likely lacks permissions.")
        print("   In Vantage Admin → Public API Client → Manage Roles, assign at least 'Skill User'.")
        print("   Then re-run this cell to fetch a NEW token and retry.")
    if ct.startswith("application/json"):
        try:
            data = r.json()
            # print a compact list (id + name if present)
            if isinstance(data, list):
                preview = [
                    {k: v for k, v in s.items() if k.lower() in ("id", "name", "title")}
                    for s in data
                ]
                print(json.dumps(preview, indent=2)[:2000])
            else:
                print(json.dumps(data, indent=2)[:2000])
        except Exception:
            print("Received JSON content but parsing failed; raw text preview:\n", r.text[:1000])
    else:
        print("Non-JSON response preview:\n", r.text[:1000])

# ---- run both steps ----
token = get_token()
call_skills(token)


✅ Token acquired (expires_in=86400s) and saved to os.environ['ABBYY_TOKEN']
Skills status: 200
[
  {
    "id": "1458486c-6a43-4323-b95b-965a0ba98849",
    "name": "arabic"
  }
]


In [2]:
# ABBYY Vantage Transaction Processing Workflow
import os
import json
import time
import requests
from pathlib import Path

# Configuration
SKILL_ID = "1458486c-6a43-4323-b95b-965a0ba98849"
BASE_URL = "https://vantage-eu.abbyy.com/api/publicapi/v1"
PDF_FOLDER = "NCAR_PDFs"
OUTPUT_FOLDER = "ABBYY_Processed"
POLL_INTERVAL = 30  # seconds between status checks
MAX_POLL_ATTEMPTS = 60  # maximum polling attempts (30 min total)

def create_transaction(pdf_file_path, token):
    """Create a transaction for processing a PDF file."""
    url = f"{BASE_URL}/transactions/launch"
    
    # Prepare the model data
    model_data = {
        "files": [
            {
                "index": 0,
                "imageProcessingOptions": {
                    "autoCrop": "Default",
                    "autoOrientation": "Default"
                },
                "registrationParameters": []
            }
        ],
        "registrationParameters": [],
        "skillParameters": []
    }
    
    headers = {
        "accept": "text/plain",
        "Authorization": f"Bearer {token}"
    }
    
    params = {"skillId": SKILL_ID}
    
    # Prepare multipart form data
    files = {
        'Model': (None, json.dumps(model_data), 'application/json'),
        'Files': (os.path.basename(pdf_file_path), open(pdf_file_path, 'rb'), 'application/pdf')
    }
    
    try:
        print(f"🚀 Creating transaction for: {os.path.basename(pdf_file_path)}")
        response = requests.post(url, headers=headers, params=params, files=files, timeout=60)
        
        # Close the file handle
        files['Files'][1].close()
        
        response.raise_for_status()
        
        # Parse response
        if response.headers.get('Content-Type', '').startswith('application/json'):
            result = response.json()
            transaction_id = result.get('transactionId')
            if transaction_id:
                print(f"✅ Transaction created: {transaction_id}")
                return transaction_id
            else:
                print(f"❌ No transaction ID in response: {result}")
                return None
        else:
            # Some APIs return transaction ID as plain text
            transaction_id = response.text.strip().strip('"')
            if transaction_id and len(transaction_id) > 10:  # Basic validation
                print(f"✅ Transaction created: {transaction_id}")
                return transaction_id
            else:
                print(f"❌ Invalid response: {response.text}")
                return None
            
    except requests.RequestException as e:
        print(f"❌ Error creating transaction for {os.path.basename(pdf_file_path)}: {e}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

def check_transaction_status(transaction_id, token):
    """Check the status of a transaction."""
    url = f"{BASE_URL}/transactions/{transaction_id}"
    headers = {
        "accept": "text/plain",
        "Authorization": f"Bearer {token}"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        
        if response.headers.get('Content-Type', '').startswith('application/json'):
            return response.json()
        else:
            # Try to parse as JSON anyway
            try:
                return json.loads(response.text)
            except:
                print(f"❌ Non-JSON response: {response.text[:200]}")
                return None
                
    except requests.RequestException as e:
        print(f"❌ Error checking transaction {transaction_id}: {e}")
        return None

def wait_for_transaction_completion(transaction_id, token):
    """Poll transaction status until completion."""
    print(f"⏳ Waiting for transaction {transaction_id} to complete...")
    
    for attempt in range(MAX_POLL_ATTEMPTS):
        status_data = check_transaction_status(transaction_id, token)
        
        if not status_data:
            print(f"❌ Failed to get status for transaction {transaction_id}")
            return None
            
        status = status_data.get('status', 'Unknown')
        print(f"📊 Transaction {transaction_id} status: {status} (attempt {attempt + 1})")
        
        if status == 'Processed':
            print(f"✅ Transaction {transaction_id} completed successfully!")
            return status_data
        elif status in ['Failed', 'Error', 'Cancelled']:
            print(f"❌ Transaction {transaction_id} failed with status: {status}")
            return status_data
        elif status in ['Processing', 'Queued', 'InProgress', 'Pending']:
            if attempt < MAX_POLL_ATTEMPTS - 1:
                print(f"⏳ Still processing... waiting {POLL_INTERVAL}s")
                time.sleep(POLL_INTERVAL)
            else:
                print(f"⏰ Maximum polling attempts reached for transaction {transaction_id}")
                return status_data
        else:
            print(f"❓ Unknown status '{status}' for transaction {transaction_id}")
            if attempt < MAX_POLL_ATTEMPTS - 1:
                time.sleep(POLL_INTERVAL)
    
    return status_data

def download_result_files(transaction_data, token, output_dir):
    """Download all result files from a completed transaction."""
    transaction_id = transaction_data.get('id')
    documents = transaction_data.get('documents', [])
    
    if not documents:
        print(f"⚠️  No documents found in transaction {transaction_id}")
        return []
    
    downloaded_files = []
    os.makedirs(output_dir, exist_ok=True)
    
    for doc_idx, document in enumerate(documents):
        result_files = document.get('resultFiles', [])
        
        for file_idx, result_file in enumerate(result_files):
            file_id = result_file.get('fileId')
            file_name = result_file.get('fileName', f'result_{doc_idx}_{file_idx}')
            
            if not file_id:
                print(f"⚠️  No file ID for result file: {file_name}")
                continue
                
            # Download URL
            download_url = f"{BASE_URL}/transactions/{transaction_id}/files/{file_id}/download"
            headers = {
                "accept": "application/octet-stream",
                "Authorization": f"Bearer {token}"
            }
            
            try:
                print(f"⬇️  Downloading: {file_name}")
                response = requests.get(download_url, headers=headers, stream=True, timeout=60)
                response.raise_for_status()
                
                # Save file
                output_path = os.path.join(output_dir, file_name)
                with open(output_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                print(f"✅ Downloaded: {output_path}")
                downloaded_files.append(output_path)
                
            except requests.RequestException as e:
                print(f"❌ Error downloading {file_name}: {e}")
            except Exception as e:
                print(f"❌ Unexpected error downloading {file_name}: {e}")
    
    return downloaded_files

def process_all_pdfs():
    """Main function to process all PDFs in the NCAR_PDFs folder."""
    # Get token (should already be available from previous cell)
    token = os.environ.get('ABBYY_TOKEN')
    if not token:
        print("❌ No ABBYY token found. Please run the token acquisition cell first.")
        return
    
    # Check if PDF folder exists
    if not os.path.exists(PDF_FOLDER):
        print(f"❌ PDF folder '{PDF_FOLDER}' not found. Please run the NCAR scraper first.")
        return
    
    # Get all PDF files
    pdf_files = [f for f in os.listdir(PDF_FOLDER) if f.lower().endswith('.pdf')]
    
    if not pdf_files:
        print(f"❌ No PDF files found in '{PDF_FOLDER}'")
        return
    
    print(f"📁 Found {len(pdf_files)} PDF files to process")
    
    # Track all transactions
    transactions = []
    
    # Create transactions for all PDFs
    print("\n🚀 Creating transactions...")
    for pdf_file in pdf_files:
        pdf_path = os.path.join(PDF_FOLDER, pdf_file)
        transaction_id = create_transaction(pdf_path, token)
        
        if transaction_id:
            transactions.append({
                'id': transaction_id,
                'pdf_file': pdf_file,
                'status': 'Created'
            })
        
        # Small delay between requests
        time.sleep(1)
    
    if not transactions:
        print("❌ No transactions were created successfully.")
        return
    
    print(f"\n✅ Created {len(transactions)} transactions")
    
    # Wait for all transactions to complete
    print("\n⏳ Waiting for all transactions to complete...")
    completed_transactions = []
    
    for transaction in transactions:
        print(f"\n--- Processing transaction for {transaction['pdf_file']} ---")
        result = wait_for_transaction_completion(transaction['id'], token)
        
        if result and result.get('status') == 'Processed':
            completed_transactions.append(result)
            transaction['status'] = 'Completed'
        else:
            transaction['status'] = 'Failed'
    
    # Download all result files
    if completed_transactions:
        print(f"\n⬇️  Downloading results from {len(completed_transactions)} completed transactions...")
        all_downloaded_files = []
        
        for transaction_data in completed_transactions:
            downloaded = download_result_files(transaction_data, token, OUTPUT_FOLDER)
            all_downloaded_files.extend(downloaded)
        
        print(f"\n🎉 Processing complete!")
        print(f"📊 Transactions created: {len(transactions)}")
        print(f"✅ Transactions completed: {len(completed_transactions)}")
        print(f"📥 Files downloaded: {len(all_downloaded_files)}")
        print(f"📁 Output folder: {OUTPUT_FOLDER}")
        
        # Summary
        print(f"\n📋 Transaction Summary:")
        for transaction in transactions:
            status_emoji = "✅" if transaction['status'] == 'Completed' else "❌" if transaction['status'] == 'Failed' else "⏳"
            print(f"  {status_emoji} {transaction['pdf_file']}: {transaction['status']}")
        
    else:
        print("❌ No transactions completed successfully.")

# Run the complete workflow
print("🌟 Starting ABBYY Vantage processing workflow...")
process_all_pdfs()

🌟 Starting ABBYY Vantage processing workflow...
📁 Found 30 PDF files to process

🚀 Creating transactions...
🚀 Creating transaction for: آلية-مشاركة-الجهات-الحكومية-في-معارض-الكتاب-الدولية-لعام-1447هـ_24939.pdf
✅ Transaction created: 6162a297-94b0-4ad5-b6bc-998ba7688825
🚀 Creating transaction for: إحلال-عبارة-وزارة-البلديات-والإسكان-محل-عبارة-وزارة-الشؤون-البلدية-والقروية-والإسكان-وإحلال-عبارة-وز_181.pdf
✅ Transaction created: 0fa4f3a8-7855-45c1-9e42-650bfc262b2a
🚀 Creating transaction for: إضافة-صلاحية-تحديد-المقابل-المالي-لعمل-المرافقين-والمرافقات-للعمالة-الوافدة-النظامية-في-المملكة-لوزي_229.pdf
✅ Transaction created: c1a93c9e-0806-4da0-82a6-ced7db4c226f
🚀 Creating transaction for: إضافة-فقرة-إلى-البند-ثالثا-من-إعادة-تنظيم-إجراءات-عملية-نشر-الأنظمة-في-الجريدة-الرسمية-لعام-1425هـ_159.pdf
✅ Transaction created: 245f360f-9d01-486c-9fcd-35f87fc4f8ed
🚀 Creating transaction for: إضافة-ممثل-لكل-من-هيئة-تنمية-البحث-والتطوير-والابتكار-وهيئة-فنون-الطهي-والهيئة-العامة-للأمن-الغذائي-_216.pdf
✅ Tr